# APIの入力データを作成、送信、出力

In [1]:
import base64
import json
import random
import time
from datetime import datetime as dt
from pathlib import Path
from typing import Dict, Optional, Union

import requests
from pyngrok import ngrok
from yamawrapper import dev_utils as du

import for_API as fa
import sql_operation as so
from app import CreateApp


### パッケージのヴァージョン確認

In [2]:
# !poetry show linedetection

### パスの設定

In [3]:
DATA_ROOT_PATH = Path(
    '/Volumes/GoogleDrive-117051946152776398938/.shortcut-targets-by-id/'
    '1EFBGYIMFr_sCrEWjIjGChD13zAFxcUzF/1_配筋検査支援メニュー/No.5_図種分類ワーク/'
    '2_データ/1_開発用/0_Ops受領データ(PDF)')
data_list = {'sonota': list((DATA_ROOT_PATH / '0_その他').glob('*.pdf')),
             'haikin': list((DATA_ROOT_PATH / '0_配筋リスト').glob('*.pdf')),
             'fusezu': list((DATA_ROOT_PATH / '0_伏図').glob('*.pdf'))}

### 解析処理の試行

In [4]:
from linedetection.line_detection import LineDetection

PROCESS = 0
# pdf_path = DATA_ROOT_PATH / Path('test/pdf_dir/2_17_ops.pdf')
# save_path = DATA_ROOT_PATH / Path('post_data_1.json')
pdf_path = DATA_ROOT_PATH / Path('test/pdf_dir/1_gd_15.pdf')
save_path = DATA_ROOT_PATH / Path('post_data.json')
if PROCESS:
    data_dict = fa.create_post_json(pdf_path, output_path=save_path)
    plan = data_dict['plan']
    plan_size = data_dict['plan_size']

    ld = LineDetection(plan, plan_size)
    result = ld.main()
    print(f'plan_size: {plan_size}\n'
          f'vector: {result["vector"]}\n'
          f'raster: {result["raster"]}\n'
          f'len(lines): {len(result["lines"])}')

### データベース、tableの作成

In [5]:
PROCESS = 0
if PROCESS:
    dbname = './api_rsps.db'
    so.create_db(dbname)

In [6]:
# local server を起動
dbname = './api_rsps.db'
ca = CreateApp(dbname)
public_url = ca.public_url
tunnels = ngrok.get_tunnels()

 * ngrok tunnel "http://fbb1-114-184-173-207.ngrok.io" -> "http://127.0.0.1:8888"
2022-09-29 11:21:55,722 INFO werkzeug WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:8888
2022-09-29 11:21:55,726 INFO werkzeug Press CTRL+C to quit


In [7]:
import random

from tqdm.notebook import tqdm

# APIにデータを送信
url = 'https://dev.line-detection.api.aoz.cloud/line-detection'
api_key = 'zSmybhmQ249cRPnAIKOWk9xbhrPc9xHvarzZ6ulr'
# webhook.site
webhook_url = public_url + '/receiver'
webhooksite_url = 'https://webhook.site/558607bc-364c-4e3c-a632-5ae171cecbcf'

n = 3
# random.seed(0)
pdf_path = random.choice(data_list['haikin'])
print(pdf_path.name)
metadata = {"pdf_name": f"{pdf_path.stem}"} # 送信するデータのメタデータ
rsps_local = fa.send_data_to_api(url=url, api_key=api_key,
                        webhook_url=webhook_url, pdf_path=pdf_path,
                        metadata=metadata)
print(f'rsps_local: {rsps_local}')
# rsps_webhooksite = fa.send_data_to_api(url=url, api_key=api_key,
#                         webhook_url=webhooksite_url, pdf_path=pdf_path,
#                         metadata=metadata)
# print(f'rsps_webhooksite: {rsps_webhooksite}')


# for pdf_path in tqdm(random.sample(data_list['haikin'], n)):
#     print(pdf_path.name)
#     metadata = {"pdf_name": f"{pdf_path.stem}"} # 送信するデータのメタデータ
#     _ = fa.send_data_to_api(url=url, api_key=api_key,
#                             webhook_url=webhook_url, pdf_path=pdf_path,
#                             metadata=metadata)

36_21_ops.pdf
rsps_local: {'message': 'Accepted', 'request_id': '3c218af8-e248-4601-a986-4c45b33a7e9e'}


In [8]:
ca.receive().keys()

dict_keys(['id', 'status', 'result', 'message', 'metadata'])

2022-09-29 11:21:59,492 INFO werkzeug 127.0.0.1 - - [29/Sep/2022 11:21:59] "POST /receiver HTTP/1.1" 200 -


In [9]:
print(f'Before tunnels:\n{ngrok.get_tunnels()}')
# Close all open ngrok tunnels
_ = [ngrok.disconnect(tnnl.public_url) for tnnl in tunnels]
print(f'After tunnels:\n{ngrok.get_tunnels()}')

Before tunnels:
[<NgrokTunnel: "http://fbb1-114-184-173-207.ngrok.io" -> "http://localhost:8888">, <NgrokTunnel: "https://fbb1-114-184-173-207.ngrok.io" -> "http://localhost:8888">]
After tunnels:
[]
